# Making observation plan (time)

## Install & Import modules

In [1]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, astroquery, requests, astropy, astropy.units, certifi, version_information".split(', ')
for package in packages:
    if not importlib.util.find_spec(package):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])

import pandas as pd
import json
import requests

from astroquery.mpc import MPC
from astropy.coordinates import SkyCoord
from astropy import units as u

import os

## Set time & condition

In [4]:
obs_date_utc = '2023-09-18'     # YYYY-MM-DD
obs_time_utc = '11:00:00'       # hh:mm:ss

mpc_code = 'P64'
elev_min = 30
time_min = 15
vmag_min = 6
vmag_max = 15
max_output = 50
sort = 'trans'      # Transit Time

## Bring & Save data

In [8]:
data_set = []

url = f"https://ssd-api.jpl.nasa.gov/sbwobs.api?sb-kind=a&mpc-code={mpc_code}&obs-time={obs_date_utc}T{obs_time_utc}&elev-min={elev_min}&time-min={time_min}&vmag-max={vmag_max}&vmag-min={vmag_min}&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={max_output}"
res = requests.get(url)
data = json.loads(res.text)

for data_line in data['data']:
    TT_hour, TT_min = data_line[3].replace('*','').split(':')
    TransTime_tmp = int(TT_hour) + int(TT_min)/60
    
    ObsTime_tmp = int(obs_time_utc[:2]) + int(obs_time_utc[3:5])/60
    
    if ObsTime_tmp - 0.5 <= TransTime_tmp and TransTime_tmp < ObsTime_tmp + 0.5:
        name = data_line[1].split('(')[1][:-1]
        try:
            result = MPC.get_ephemeris(name, location = mpc_code, start = obs_date_utc, step = '1h')[round(TransTime_tmp)]
            
            c = SkyCoord(ra = result[1]*u.degree, dec = result[2]*u.degree)
            Ra,Dec = c.to_string('hmsdms').split(' ')
            Ra = Ra.replace('h',':').replace('m',':').replace('s','')
            Dec = Dec.replace('d',':').replace('m',':').replace('s','').replace('+','')
            
            Vmag = result[7]
            Azi = result[10]
            Alt = result[11]

            if int(Alt) >= elev_min:
                data_set.append([name, Ra, Dec, Vmag, Azi, Alt, f'{TT_hour}:{TT_min}'])
        except:
            continue

## Show data

In [10]:
if len(data_set)!=0:
    df = pd.DataFrame.from_dict(data_set)
    df = df.set_axis(['Name', 'RA', 'Dec', 'V-mag', 'Azimuth', 'Altitude', 'Transit time(UTC)'], axis = 1)
    df

3


,Name,RA,Dec,V-mag,Azimuth,Altitude,Transit time(UTC)
0,A894 VC,19:09:07.1,-00:40:26,11.6,183,52,10:56
1,A922 OD,19:20:41.6,-06:22:02,14.8,178,46,11:09
2,A922 MA,19:36:27,-08:24:18,14.6,173,44,11:24


## Save as xml file

In [6]:
ObsPlan = [[30,'r',1,10],[60,'r',1,10],[90,'r',1,10],[30,'r',2,10],[60,'r',2,10],[90,'r',2,10],]   # ExpTime, Filter, Bin, TotalExpCnt
Sequence_set = []

for asteroid_data in data_set:
  Targetname = asteroid_data[0].replace(' ','') + '_' + obs_date_utc.replace('-','')[2:] + '_' + obs_time_utc.replace(':','')[:4]    # 제목 형식: Name_yymmdd_hhmm(UTC)
  
  RAHours, RAMinutes, RASeconds = asteroid_data[1].split(':');RASeconds = RASeconds[:2]   # hh, mm, ss
  DecDegrees, DecMinutes, DecSeconds = asteroid_data[2].split(':');DecSeconds = DecSeconds[:2]    # hh, mm, ss

  pre = """<?xml version="1.0" encoding="utf-8"?>\n"""
  CSlist = f"""<CaptureSequenceList xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" TargetName="{Targetname}" Mode="ROTATE" RAHours="{RAHours}" RAMinutes="{RAMinutes}" RASeconds="{RASeconds}" DecDegrees="{DecDegrees}" DecMinutes="{DecMinutes}" DecSeconds="{DecSeconds}" Rotation="0" Delay="0" SlewToTarget="true" AutoFocusOnStart="true" CenterTarget="true" RotateTarget="false" StartGuiding="true" AutoFocusOnFilterChange="false" AutoFocusAfterSetTime="false" AutoFocusSetTime="30" AutoFocusAfterSetExposures="false" AutoFocusSetExposures="10" AutoFocusAfterTemperatureChange="false" AutoFocusAfterTemperatureChangeAmount="5" AutoFocusAfterHFRChange="false" AutoFocusAfterHFRChangeAmount="10">\n"""
  xmlDoc = pre + CSlist   # xml 문서

  for Obs in ObsPlan:   # 관측 계획(알짜 요소)
    CapSeq = f"""  <CaptureSequence>
    <Enabled>true</Enabled>
    <ExposureTime>{Obs[0]}</ExposureTime>
    <ImageType>LIGHT</ImageType>
    <FilterType>
      <Name>{Obs[1]}</Name>
      <FocusOffset>0</FocusOffset>
      <Position>3</Position>
      <AutoFocusExposureTime>-1</AutoFocusExposureTime>
      <AutoFocusFilter>false</AutoFocusFilter>
      <FlatWizardFilterSettings>
        <FlatWizardMode>DYNAMICEXPOSURE</FlatWizardMode>
        <HistogramMeanTarget>0.5</HistogramMeanTarget>
        <HistogramTolerance>0.1</HistogramTolerance>
        <MaxFlatExposureTime>30</MaxFlatExposureTime>
        <MinFlatExposureTime>0.01</MinFlatExposureTime>
        <StepSize>0.1</StepSize>
        <MaxAbsoluteFlatDeviceBrightness>1</MaxAbsoluteFlatDeviceBrightness>
        <MinAbsoluteFlatDeviceBrightness>0</MinAbsoluteFlatDeviceBrightness>
        <FlatDeviceAbsoluteStepSize>1</FlatDeviceAbsoluteStepSize>
      </FlatWizardFilterSettings>
      <AutoFocusBinning>
        <X>1</X>
        <Y>1</Y>
      </AutoFocusBinning>
      <AutoFocusGain>-1</AutoFocusGain>
      <AutoFocusOffset>-1</AutoFocusOffset>
    </FilterType>
    <Binning>
      <X>{Obs[2]}</X>
      <Y>{Obs[2]}</Y>
    </Binning>
    <Gain>-1</Gain>
    <Offset>-1</Offset>
    <TotalExposureCount>{Obs[3]}</TotalExposureCount>
    <ProgressExposureCount>0</ProgressExposureCount>
    <Dither>true</Dither>
    <DitherAmount>1</DitherAmount>
  </CaptureSequence>\n"""
    xmlDoc += CapSeq

  end = f"""  <Coordinates>
    <RA>{int(RAHours) + int(RAMinutes)/60 + int(RASeconds)/3600}</RA>
    <Dec>{int(DecDegrees) + int(DecMinutes)/60 + int(DecMinutes)/3600}</Dec>
    <Epoch>J2000</Epoch>
  </Coordinates>
  <NegativeDec>false</NegativeDec>
</CaptureSequenceList>\n"""
  xmlDoc += end
  
  Sequence_set.append([Targetname, xmlDoc])   # 제목, xml 문서 내용


In [7]:
for Sequence in Sequence_set:
    print(f'Target Name : {Targetname}')
    print(xmlDoc,'\n\n')

Target Name : A922MA_230918_1100
<?xml version="1.0" encoding="utf-8"?>
<CaptureSequenceList xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" TargetName="A922MA_230918_1100" Mode="ROTATE" RAHours="19" RAMinutes="36" RASeconds="27" DecDegrees="-08" DecMinutes="24" DecSeconds="18" Rotation="0" Delay="0" SlewToTarget="true" AutoFocusOnStart="true" CenterTarget="true" RotateTarget="false" StartGuiding="true" AutoFocusOnFilterChange="false" AutoFocusAfterSetTime="false" AutoFocusSetTime="30" AutoFocusAfterSetExposures="false" AutoFocusSetExposures="10" AutoFocusAfterTemperatureChange="false" AutoFocusAfterTemperatureChangeAmount="5" AutoFocusAfterHFRChange="false" AutoFocusAfterHFRChangeAmount="10">
  <CaptureSequence>
    <Enabled>true</Enabled>
    <ExposureTime>30</ExposureTime>
    <ImageType>LIGHT</ImageType>
    <FilterType>
      <Name>r</Name>
      <FocusOffset>0</FocusOffset>
      <Position>3</Position>
      <AutoFocusExposureTim